In [1]:
import numpy as np 
import random

In [2]:
import pandas as pd
import numpy as np
file=pd.read_csv('C:\\Users\\zhou\\Desktop\\train.csv')
file1=pd.read_csv('C:\\Users\\zhou\\Desktop\\test.csv')
file2=pd.read_csv('C:\\Users\\zhou\\Desktop\\gender_submission.csv')

In [3]:
data_train=file.loc[0:,['Pclass','Sex','Age','SibSp','Parch','Fare']]
data_train_label=file.loc[0:,'Survived']
data_test=file1.loc[0:,['Pclass','Sex','Age','SibSp','Parch','Fare']]
data_test_label=file2.iloc[0:,0:]

In [4]:
count=0
for i in data_train.loc[0:,'Sex']:
    count+=1
    if i=='female':
        data_train.loc[0:,'Sex'][count-1]=0
    if i=='male':
        data_train.loc[0:,'Sex'][count-1]=1
count=0
for i in data_test.loc[0:,'Sex']:
    count+=1
    if i=='female':
        data_test.loc[0:,'Sex'][count-1]=0
    if i=='male':
        data_test.loc[0:,'Sex'][count-1]=1

In [5]:
data_others=np.array(data_train)[0:,[0,1,3,4,5]]
data_age=np.array(data_train)[0:,[2]]
data_test_others=np.array(data_test)[0:,[0,1,3,4,5]]
data_test_age=np.array(data_test)[0:,[2]]

In [6]:
nan_index_list=[]
for i in range(891):
    if pd.isnull(data_age[i][0]):
        nan_index_list.append(i)  
nan_test_index_list=[]
for i in range(418):
    if pd.isnull(data_test_age[i][0]):
        nan_test_index_list.append(i) 

In [7]:
#归一化处理
data_others=(data_others-data_others.min(axis=0))/(data_others.max(axis=0)-data_others.min(axis=0))
data_test_others=(data_test_others-data_test_others.min(axis=0))/(data_test_others.max(axis=0)-data_test_others.min(axis=0))

C:\Users\zhou\anaconda3\lib\site-packages\numpy\core\_methods.py:34: RuntimeWarning: invalid value encountered in reduce
  return umr_minimum(a, axis, None, out, keepdims, initial, where)
C:\Users\zhou\anaconda3\lib\site-packages\numpy\core\_methods.py:30: RuntimeWarning: invalid value encountered in reduce
  return umr_maximum(a, axis, None, out, keepdims, initial, where)


In [8]:
#利用knn算法补充age的缺失值
def knn(nums,data_others,k,data_age):
    data_size=data_others.shape[0]
    diffmat=np.tile(data_others[nums],(data_size,1))-data_others
    diffmat_sqr=diffmat**2
    sqrdiff=diffmat_sqr.sum(axis=1)
    distance_set=sqrdiff**0.5
    distance_sort=np.argsort(distance_set)
    list_near_average=[]
    for i in range(k):
        list_near_average.append(distance_sort[i])
    average=[]
    for i in list_near_average:
        average.append(data_age[i][0])
    average_new=[]
    for i in average:
        if pd.notnull(i):
            average_new.append(i)
    return sum(average_new)/len(average_new)
for nums in nan_index_list:
    data_train.loc[0:,'Age'][nums]=knn(nums,data_others,10,data_age)
for nums1 in nan_test_index_list:
    data_test.loc[0:,'Age'][nums1]=knn(nums1,data_test_others,10,data_test_age)

In [9]:
data_train=np.array(data_train)
data_train_label=np.array(data_train_label)

In [10]:
data_test=np.array(data_test)
data_test_label=np.array(data_test_label)
data_test_label=data_test_label[0:,1]

In [11]:
data_train=(data_train-data_train.min(axis=0))/(data_train.max(axis=0)-data_train.min(axis=0))

In [12]:
data_test=(data_test-data_test.min(axis=0))/(data_test.max(axis=0)-data_test.min(axis=0))

In [13]:
for i in range(len(data_train_label)):
    if data_train_label[i]==0:
        data_train_label[i]=-1

In [14]:
for i in range(len(data_test_label)):
    if data_test_label[i]==0:
        data_test_label[i]=-1

In [15]:
data_test

array([[1.0, 1.0, 0.45272319662402744, 0.0, 0.0, 0.015281580671177828],
       [1.0, 0.0, 0.6175656072794409, 0.125, 0.0, 0.013663090060062943],
       [0.5, 1.0, 0.815376500065937, 0.0, 0.0, 0.018908740708122825],
       ...,
       [1.0, 1.0, 0.5054727680337597, 0.0, 0.0, 0.014151057562208049],
       [1.0, 1.0, 0.37359883950942896, 0.0, 0.0, 0.015712553569072387],
       [1.0, 1.0, 0.2664512725834102, 0.125, 0.1111111111111111,
        0.04364049521284361]], dtype=object)

In [16]:
#高斯核函数
def rbf(train,sigma):
    list_vector=[]
    m,n=train.shape
    for num in range(len(train)):
        vector1=((train-train[num])**2).sum(axis=1)
        vector2=vector1**0.5
        vector=vector2/(2*(sigma**2))
        vector=-vector.reshape(1,m)
        vector=np.exp(vector.astype('float'))        
        list_vector.append(np.exp(vector))
    train=np.vstack(list_vector)
    return train
   

In [17]:
data_train=rbf(data_train,1)

In [18]:
data_test=rbf(data_test,1)

In [76]:
def hinge(train,train_label,alpha,maxiter,learnrate):
    m,n=train.shape
    train_label=train_label.reshape((m,1))
    w=np.random.randn(n,1)
    b=0
    Iter=0
    while Iter<maxiter:
        f_x=train.dot(w)+b
        E_x=1-f_x*train_label
        E_x1=(E_x+abs(E_x))/2
        for i in range(len(E_x1)):
            if E_x1[i][0]>0:
                E_x1[i][0]=1
            if E_x1[i][0]==0:
                pass
        #梯度下降
        w=w+learnrate*((((E_x*train_label*train).T).dot(E_x1))+alpha*w)
        b=b+learnrate*(train_label.T.dot(E_x1))
        Iter+=1
    return w,b

In [64]:
train=data_train[0:500,:]

In [67]:
train_label=data_train_label[0:500]

In [111]:
w,b=hinge(train,train_label,0.3,30,0.000001)

In [113]:
count=0
count1=0
for sample in data_train[500:,:]:
    count+=1
    fx=sample.dot(w)+b
    if fx[0][0]>0:
        a=1
    else:
        a=-1
    if a==data_train_label[500+count-1]:
        count1+=1
print(count1/count)

0.7544757033248082
